In [1]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d, griddata
from itertools import product
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
import glob
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')


pd.set_option('display.max_rows', 200)

In [2]:
cmsData = pd.read_pickle('../AuxInfo/CMS-EXO-20-004-data/DMsimp_spin-0_Monojet_DF.pcl')

In [3]:
recastData1 = pd.read_pickle('../../MonoXSMS-version2/run_08/pseudoscalar_450_1_cms_exo_20_004.pcl')
recastData1['version'] = 'version2-single'

recastData2 = pd.read_pickle('../scanResults_pseudoscalar_single.pcl')
recastData2['version'] = 'version3-single'

recastData3 = pd.read_pickle('../../MonoXSMS-version2/DMSimp_pseudoscalar_0j/Events/run_03/tag_1_cms_exo_20_004.pcl')
recastData3['version'] = 'version2-match'

recastData4 = pd.read_pickle('../scanResults-pseudoscalar-cms.pcl')
recastData4['version'] = 'version3-match'

recastData = pd.concat([recastData1, recastData2, recastData3, recastData4], ignore_index=True)

In [4]:
models = []
mCols = ['Coupling', 'Mode', '$m_{med}$', '$m_{DM}$']
for row in recastData[mCols].values:
    m = dict(zip(mCols, row.tolist()))
    if m not in models:
        models.append(m)

print('%i models loaded.' %len(models))

24 models loaded.


In [5]:
parameters = {label : list(set([m[label] for m in models])) for label in mCols}
for k,v in parameters.items():
    if len(v) == 1:
        parameters[k] = v[0]
print(parameters)

textstr = '\n'.join([r'%s = %s' %(k,v) for k,v in parameters.items() if k not in ['$m_{med}$','$m_{DM}$']])
print(textstr)

{'Coupling': 'Pseudoscalar', 'Mode': 'DM+QCDjets', '$m_{med}$': [800.0, 450.0, 100.0, 200.0, 10.0, 300.0, 400.0, 50.0, 500.0, 600.0, 700.0, 350.0], '$m_{DM}$': [1.0, 225.0, 4.0, 6.0, 40.0, 200.0, 45.0, 50.0, 275.0, 20.0, 22.0, 55.0, 150.0]}
Coupling = Pseudoscalar
Mode = DM+QCDjets


In [6]:
paramFilter = {'Coupling': 'Pseudoscalar',
    'Mode' : 'DM+QCDjets',
    '$m_{med}$' : 450.,
    '$m_{DM}$' : 1.,
    '$g_{DM}$' : 1.,
    'Data-takingperiod' : 2017
}

textstr = '\n'.join([r'%s = %s' %(k,v) for k,v in paramFilter.items()])
recastData = recastData.loc[(recastData[list(paramFilter)] == pd.Series(paramFilter)).all(axis=1)]
cmsData = cmsData.loc[(cmsData[list(paramFilter)] == pd.Series(paramFilter)).all(axis=1)]

In [7]:
recastData.sort_values(['Coupling','Mode','$m_{med}$','$m_{DM}$','Data-takingperiod'],inplace=True,
                ascending=[False,False,True,True,False],ignore_index=True)  
cmsData.sort_values(['Coupling','Mode','$m_{med}$','$m_{DM}$','Data-takingperiod'],inplace=True,
                ascending=[False,False,True,True,False],ignore_index=True)  

In [8]:
muCMS = np.array(cmsData['$\mu^{UL}_{obs}$'])

In [9]:
recastData['UL ratio'] = muCMS/recastData['$\\mu^{UL}_{obs}$']

In [10]:
recastData.T

,0,1,2,3
Data-takingperiod,2017,2017,2017,2017
Luminosity (1/fb),41.5,41.5,41.5,41.5
bin_250.0_280.0,1065.606135,1074.207588,993.142039,1007.318495
bin_250.0_280.0_ErrorPlus,45.109991,57.186463,56.56314,52.04036
bin_250.0_280.0_ErrorMinus,45.109991,57.186463,56.56314,52.04036
bin_280.0_310.0,755.230309,805.442356,668.659335,650.840617
bin_280.0_310.0_ErrorPlus,34.437551,47.711296,44.31238,34.145823
bin_280.0_310.0_ErrorMinus,34.437551,47.711296,44.31238,34.145823
bin_310.0_340.0,650.419891,507.445348,519.188263,583.349127
bin_310.0_340.0_ErrorPlus,35.940667,32.393783,35.693225,80.343982
